In [1]:
import os
# check if session has spark
os.listdir('/content/')
#os.listdir()

['.config', 'sample_data']

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,910 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,183 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricte

In [5]:
# one time thing
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# Start a Spark session
import findspark
findspark.init()

# Create a SparkSession
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
#sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("MyApp").getOrCreate()
spark.sparkContext.setCheckpointDir("/content/drive/My Drive/USPS/spark_checkpoint")

os.chdir('/content/drive/My Drive/USPS')



In [7]:
# Read parquet file to DF
folder1 = "/content/drive/My Drive/USPS/parq_files/usps_0501.parquet"
folder2 = "/content/drive/My Drive/USPS/parq_files/usps_0502.parquet"
folder3 = "/content/drive/My Drive/USPS/parq_files/usps_0503.parquet"
parc_folders = [folder1,folder2,folder3]
df = spark.read.parquet(*parc_folders)

#Cleaning -----
# data from 0503 read ServiceType code as int so it removed 0s to the left;adding those back
df = df.withColumn('ServiceTypeCode', lpad(col('ServiceTypeCode'), 3, '0'))

# Get the list of columns that contain 'zipcode'
zipcode_columns = [col for col in df.columns if 'zipcode' in col]

# Define a function to add leading zeros
def add_leading_zeros(col):
    return lpad(col, 5, '0')

# Apply the function to all zipcode columns
for col in zipcode_columns:
    df = df.withColumn(col, add_leading_zeros(df[col]))

# recode CMRA flag
df = df.withColumn(
    'CRID_cmra_flag',
    when(df['CRID_cmra_flag'] == 'Y', True)
    .when(df['CRID_cmra_flag'].isin(['N', 'X']), False)
    .otherwise(None)
)

# recode delinked nulls
df = df.withColumn("delinked", when(df["delinked"].isNull(), False).otherwise(df["delinked"]))



In [8]:
# create potential fraud column
df = df.withColumn(
    "potential_fraud",
    when(df["delinked"] == True, True)
    .when(df["icr_or_manifest"] == False, True)
    .when(df["symbology_mismatch"] == True, True)
    .when(df["firstzip_startzip_diff"] == True, True)
    .otherwise(False)
)

In [9]:
total_rows = df.count()
total_rows #61,256,141

61256141

In [10]:
df.groupby('potential_fraud').count().show()

+---------------+--------+
|potential_fraud|   count|
+---------------+--------+
|           true|21918680|
|          false|39337461|
+---------------+--------+



In [12]:
n_non_fraud = 39337461

In [13]:
# Select records with "potential fraud" == True
fraud_records = df.filter(df["potential_fraud"] == True)
n_fraud = fraud_records.count()
# Select a random assortment of records with "potential fraud" == False
non_fraud_records = df.filter(df["potential_fraud"] == False).sample(fraction = n_fraud/n_non_fraud)# total of non_fraud

# Combine the two sets of records
working_dataset = fraud_records.union(non_fraud_records)

In [14]:
working_dataset = working_dataset.coalesce(1)

In [ ]:
working_dataset.write.csv("/content/drive/My Drive/USPS/working_dataset.csv")

In [ ]:
subset_100k_noW= df.sample(fraction=0.1)
subset_100k_noW = subset_100k_noW.coalesce(1)
subset_100k_noW.write.csv("/content/drive/My Drive/USPS/subset_100k_noW.csv")

In [ ]:
subset_10perc_W= working_dataset.sample(fraction=0.1)
subset_10perc_W = subset_10perc_W.coalesce(1)
subset_10perc_W.write.csv("/content/drive/My Drive/USPS/subset_10perc_W.csv")

In [ ]:
subset_1perc_W= working_dataset.sample(fraction=0.01)
subset_1perc_W = subset_1perc_W.coalesce(1)
subset_1perc_W.write.csv("/content/drive/My Drive/USPS/subset_1perc_W.csv")